##Загрузка данных

In [1]:
!wget "https://files.grouplens.org/datasets/movielens/ml-100k.zip"

--2022-03-07 05:30:47--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  3.47MB/s    in 1.4s    

2022-03-07 05:30:50 (3.47 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



In [2]:
import os
import zipfile


data_r = zipfile.ZipFile('ml-100k.zip', 'r')
data_r.printdir()

# extract the contents
data_r.extractall()

File Name                                             Modified             Size
ml-100k/                                       2016-01-29 14:26:28            0
ml-100k/allbut.pl                              2000-07-19 16:09:28          716
ml-100k/mku.sh                                 2000-07-19 16:09:28          643
ml-100k/README                                 2016-01-29 14:26:28         6750
ml-100k/u.data                                 2000-07-19 16:09:30      1979173
ml-100k/u.genre                                2000-07-19 16:09:30          202
ml-100k/u.info                                 2000-07-19 16:09:30           36
ml-100k/u.item                                 2000-07-19 16:09:30       236344
ml-100k/u.occupation                           2000-07-19 16:09:30          193
ml-100k/u.user                                 2000-07-19 16:09:30        22628
ml-100k/u1.base                                2001-03-08 12:33:08      1586544
ml-100k/u1.test                         

In [3]:
import pandas as pd

column_names_rating = ['user_id', 'movie_id','rating','timestamp']
df_rating = pd.read_csv('ml-100k/u.data',sep='\t', header=None, names = column_names_rating)
df_rating.head(2)

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742


In [4]:
column_names_movie = ['movie_id', 'movie_title', 'release_date', 'video_release_date',
                      'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation',
                      'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                      'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                      'Thriller', 'War', 'Western',
                      ]
df_movie = pd.read_csv('ml-100k/u.item',sep='|', header=None, names = column_names_movie, encoding="ISO-8859-1")
df_movie.head(2)

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0


##Средствами Pandas, используя dataframe ratings, найдите id пользователя, поставившего больше всего оценок

In [5]:
# 1 способ
df_tmp = df_rating.groupby('user_id').count().sort_values(by='movie_id', ascending=False)

# 2 способ
df_tmp =df_rating.value_counts('user_id')

df_tmp.index[0]

405

##Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь 

In [6]:
user = df_tmp.index[0]
df_rating = df_rating[df_rating['user_id']==user]
df_rating

,user_id,movie_id,rating,timestamp
12276,405,56,4,885544911
12383,405,592,1,885548670
12430,405,1582,1,885548670
12449,405,171,1,885549544
12460,405,580,1,885547447
...,...,...,...,...
98956,405,375,1,885546835
98978,405,445,4,885548435
99148,405,1246,1,885547735
99465,405,196,1,885546112


##Добавьте к датафрейму из задания 3 столбцы:
- По жанрам. Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет
- столбцы с общим количеством оценок от всех пользователей на фильм и суммарной оценкой от всех пользователей


In [7]:
column_names_movie_genre = ['movie_id', 'unknown', 'Action', 'Adventure', 'Animation',
                            'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                            'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                            'Thriller', 'War', 'Western',
                            ]

df_rating_genre = df_rating.merge(df_movie[column_names_movie_genre], left_on='movie_id', right_on='movie_id')
df_rating_genre.head(2)

,user_id,movie_id,rating,timestamp,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,405,56,4,885544911,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,405,592,1,885548670,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [8]:
column_names_rating = ['user_id', 'movie_id','num_rating','timestamp']
df_rating = pd.read_csv('ml-100k/u.data',sep='\t', header=None, names = column_names_rating)
df_number_of_ratings = df_rating.groupby('movie_id').count()
df_number_of_ratings.head(2)

,user_id,num_rating,timestamp
movie_id,,,
1,452,452,452
2,131,131,131


In [9]:
df_rating_genre_and_numrating = df_rating_genre.merge(df_number_of_ratings[['num_rating']], left_on='movie_id', right_index=True)
df_rating_genre_and_numrating.head(2)

,user_id,movie_id,rating,timestamp,unknown,Action,Adventure,Animation,Children's,Comedy,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,num_rating
0,405,56,4,885544911,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,394
1,405,592,1,885548670,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,9


In [10]:
column_names_rating = ['user_id', 'movie_id','sum_rating','timestamp']
df_rating = pd.read_csv('ml-100k/u.data',sep='\t', header=None, names = column_names_rating)
df_sum_of_ratings = df_rating.groupby('movie_id').agg(sum)
df_sum_of_ratings.head(2)

,user_id,sum_rating,timestamp
movie_id,,,
1,215609,1753,399028021059
2,64453,420,115727673079


In [11]:
df_rating_genre_and_numrating_and_sumrating = df_rating_genre_and_numrating.merge(
    df_sum_of_ratings[['sum_rating']], left_on='movie_id', right_index=True
    )
df_rating_genre_and_numrating_and_sumrating.head(2)

,user_id,movie_id,rating,timestamp,unknown,Action,Adventure,Animation,Children's,Comedy,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,num_rating,sum_rating
0,405,56,4,885544911,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,394,1600
1,405,592,1,885548670,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,9,30


##Сформируйте X_train, X_test, y_train, y_test


In [12]:
from sklearn.model_selection import train_test_split

X = df_rating_genre_and_numrating_and_sumrating.drop(columns=['user_id', 'movie_id', 'rating'], axis=1)
y = df_rating_genre_and_numrating_and_sumrating['rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##Возьмите модель линейной регрессии (или любую другую для задачи регрессии)  и обучите ее на фильмах

In [13]:
from sklearn.linear_model import Ridge

model = Ridge()
model.fit(X_train,y_train)


Ridge()

##Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии

In [14]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, model.predict(X_test))

1.5389848421165504

Для того чтобы оценить качество модели нужно с чем-то ее сравнить. Предлагаю за базовое решение принять педсказание по среднеарифметическому значению от тренировочной выборки. Расчитаем:

In [15]:
import numpy as np

base_line = np.zeros(len(y_test)) + y_train.mean()
base_line
mean_squared_error(y_test, base_line)

2.0220068210871593

Как можно заметить среднеквадратическая ошибка на базовой модели выше чем на построенной нами. Это говорит о том, что пресказательная способность построеной нами модели выше, чем заданный base line.

## Установка PySpark

In [16]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

In [17]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 26 kB/s 
     |████████████████████████████████| 198 kB 50.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=63eb7568960ed4548e1458e33b82f024d46c964f6b87cc915dca32faa9955ec9
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [18]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

##Загрузить данные в spark

In [24]:
df = spark.read.csv('ml-100k/u.data', inferSchema=True, header=False, sep='\t')
df = df.withColumnRenamed("_c0", "user_id").withColumnRenamed("_c1", "movie_id") \
        .withColumnRenamed("_c2", "rating").withColumnRenamed("_c3", "timestamp")
df.show(5)

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|    196|     242|     3|881250949|
|    186|     302|     3|891717742|
|     22|     377|     1|878887116|
|    244|      51|     2|880606923|
|    166|     346|     1|886397596|
+-------+--------+------+---------+
only showing top 5 rows



##Средствами спарка вывести среднюю оценку для каждого фильма

In [25]:
df_mean = df.groupBy('movie_id').mean('rating')
df_mean.show(5)

+--------+------------------+
|movie_id|       avg(rating)|
+--------+------------------+
|     496| 4.121212121212121|
|     471|3.6108597285067874|
|     463| 3.859154929577465|
|     148|          3.203125|
|    1342|               2.5|
+--------+------------------+
only showing top 5 rows



##Посчитайте средствами спарка среднюю оценку для каждого жанра

In [27]:
df_movie = spark.read.csv('ml-100k/u.item',sep='|', inferSchema=True, header=False, encoding="ISO-8859-1")
df_movie = df_movie.withColumnRenamed("_c0", "movie_id").withColumnRenamed("_c1", "movie_title") \
            .withColumnRenamed("_c2", "release_date").withColumnRenamed("_c3", "video_release_date") \
            .withColumnRenamed("_c4", "IMDb_URL").withColumnRenamed("_c5", "unknown") \
            .withColumnRenamed("_c6", "Action").withColumnRenamed("_c7", "Adventure") \
            .withColumnRenamed("_c8", "Animation").withColumnRenamed("_c9", "Children\'s") \
            .withColumnRenamed("_c10", "Comedy").withColumnRenamed("_c11", "Crime") \
            .withColumnRenamed("_c12", "Documentary").withColumnRenamed("_c13", "Drama") \
            .withColumnRenamed("_c14", "Fantasy").withColumnRenamed("_c15", "Film-Noir") \
            .withColumnRenamed("_c16", "Horror").withColumnRenamed("_c17", "Musical") \
            .withColumnRenamed("_c18", "Mystery").withColumnRenamed("_c19", "Romance") \
            .withColumnRenamed("_c20", "Sci-Fi").withColumnRenamed("_c21", "Thriller") \
            .withColumnRenamed("_c22", "War").withColumnRenamed("_c23", "Western")
df_movie.show(5)


+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|      movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1| Toy Story (1995)| 01-Jan-1995|              null|http://us.imdb.co...|      0|     0|        0|        1|         1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|       2| GoldenEye (1995)| 01-Jan-1995|              n

In [35]:
df_unknown = df.join(df_movie, df.movie_id==df_movie.movie_id, 'inner').select(df_movie.unknown, df.rating)
df_unknown = df_unknown.groupBy('unknown').mean('rating')
df_unknown.filter(df_unknown.unknown == 1).show()

+-------+-----------+
|unknown|avg(rating)|
+-------+-----------+
|      1|        3.2|
+-------+-----------+



In [36]:
df_action = df.join(df_movie, df.movie_id==df_movie.movie_id, 'inner').select(df_movie.Action, df.rating)
df_action = df_action.groupBy('Action').mean('rating')
df_action.filter(df_action.Action == 1).show()

+------+-----------------+
|Action|      avg(rating)|
+------+-----------------+
|     1|3.480245417953027|
+------+-----------------+



In [37]:
df_adventure = df.join(df_movie, df.movie_id==df_movie.movie_id, 'inner').select(df_movie.Adventure, df.rating)
df_adventure = df_adventure.groupBy('Adventure').mean('rating')
df_adventure.filter(df_adventure.Adventure == 1).show()

+---------+-----------------+
|Adventure|      avg(rating)|
+---------+-----------------+
|        1|3.503526503308369|
+---------+-----------------+



In [38]:
df_animation = df.join(df_movie, df.movie_id==df_movie.movie_id, 'inner').select(df_movie.Animation, df.rating)
df_animation = df_animation.groupBy('Animation').mean('rating')
df_animation.filter(df_animation.Animation == 1).show()

+---------+------------------+
|Animation|       avg(rating)|
+---------+------------------+
|        1|3.5766990291262135|
+---------+------------------+



In [39]:
df_children = df.join(df_movie, df.movie_id==df_movie.movie_id, 'inner').select(df_movie['Children\'s'], df.rating)
df_children = df_children.groupBy('Children\'s').mean('rating')
df_children.filter(df_children['Children\'s'] == 1).show()

+----------+------------------+
|Children's|       avg(rating)|
+----------+------------------+
|         1|3.3532442216652742|
+----------+------------------+



In [40]:
df_comedy = df.join(df_movie, df.movie_id==df_movie.movie_id, 'inner').select(df_movie.Comedy, df.rating)
df_comedy = df_comedy.groupBy('Comedy').mean('rating')
df_comedy.filter(df_comedy.Comedy == 1).show()

+------+------------------+
|Comedy|       avg(rating)|
+------+------------------+
|     1|3.3940734781442745|
+------+------------------+



In [41]:
df_crime = df.join(df_movie, df.movie_id==df_movie.movie_id, 'inner').select(df_movie.Crime, df.rating)
df_crime = df_crime.groupBy('Crime').mean('rating')
df_crime.filter(df_crime.Crime == 1).show()

+-----+------------------+
|Crime|       avg(rating)|
+-----+------------------+
|    1|3.6322780881440098|
+-----+------------------+



In [42]:
df_documentary = df.join(df_movie, df.movie_id==df_movie.movie_id, 'inner').select(df_movie.Documentary, df.rating)
df_documentary = df_documentary.groupBy('Documentary').mean('rating')
df_documentary.filter(df_documentary.Documentary == 1).show()

+-----------+------------------+
|Documentary|       avg(rating)|
+-----------+------------------+
|          1|3.6728232189973613|
+-----------+------------------+



In [43]:
df_drama = df.join(df_movie, df.movie_id==df_movie.movie_id, 'inner').select(df_movie.Drama, df.rating)
df_drama = df_drama.groupBy('Drama').mean('rating')
df_drama.filter(df_drama.Drama == 1).show()

+-----+------------------+
|Drama|       avg(rating)|
+-----+------------------+
|    1|3.6873793708484772|
+-----+------------------+



In [44]:
df_fantasy = df.join(df_movie, df.movie_id==df_movie.movie_id, 'inner').select(df_movie.Fantasy, df.rating)
df_fantasy = df_fantasy.groupBy('Fantasy').mean('rating')
df_fantasy.filter(df_fantasy.Fantasy == 1).show()

+-------+------------------+
|Fantasy|       avg(rating)|
+-------+------------------+
|      1|3.2152366863905324|
+-------+------------------+



In [45]:
df_film_noir = df.join(df_movie, df.movie_id==df_movie.movie_id, 'inner').select(df_movie['Film-Noir'], df.rating)
df_film_noir = df_film_noir.groupBy('Film-Noir').mean('rating')
df_film_noir.filter(df_film_noir['Film-Noir'] == 1).show()

+---------+------------------+
|Film-Noir|       avg(rating)|
+---------+------------------+
|        1|3.9215233698788228|
+---------+------------------+



In [46]:
df_horror = df.join(df_movie, df.movie_id==df_movie.movie_id, 'inner').select(df_movie.Horror, df.rating)
df_horror = df_horror.groupBy('Horror').mean('rating')
df_horror.filter(df_horror.Horror == 1).show()

+------+------------------+
|Horror|       avg(rating)|
+------+------------------+
|     1|3.2903893172841827|
+------+------------------+



In [47]:
df_musical = df.join(df_movie, df.movie_id==df_movie.movie_id, 'inner').select(df_movie.Musical, df.rating)
df_musical = df_musical.groupBy('Musical').mean('rating')
df_musical.filter(df_musical.Musical == 1).show()

+-------+-----------------+
|Musical|      avg(rating)|
+-------+-----------------+
|      1|3.521396851029471|
+-------+-----------------+



In [48]:
df_mystery = df.join(df_movie, df.movie_id==df_movie.movie_id, 'inner').select(df_movie.Mystery, df.rating)
df_mystery = df_mystery.groupBy('Mystery').mean('rating')
df_mystery.filter(df_mystery.Mystery == 1).show()

+-------+----------------+
|Mystery|     avg(rating)|
+-------+----------------+
|      1|3.63813155386082|
+-------+----------------+



In [49]:
df_romance = df.join(df_movie, df.movie_id==df_movie.movie_id, 'inner').select(df_movie.Romance, df.rating)
df_romance = df_romance.groupBy('Romance').mean('rating')
df_romance.filter(df_romance.Romance == 1).show()

+-------+-----------------+
|Romance|      avg(rating)|
+-------+-----------------+
|      1|3.621704948358255|
+-------+-----------------+



In [50]:
df_sci_fi = df.join(df_movie, df.movie_id==df_movie.movie_id, 'inner').select(df_movie['Sci-Fi'], df.rating)
df_sci_fi = df_sci_fi.groupBy('Sci-Fi').mean('rating')
df_sci_fi.filter(df_sci_fi['Sci-Fi'] == 1).show()

+------+------------------+
|Sci-Fi|       avg(rating)|
+------+------------------+
|     1|3.5607227022780834|
+------+------------------+



In [52]:
df_thriller = df.join(df_movie, df.movie_id==df_movie.movie_id, 'inner').select(df_movie.Thriller, df.rating)
df_thriller = df_thriller.groupBy('Thriller').mean('rating')
df_thriller.filter(df_thriller.Thriller == 1).show()

+--------+------------------+
|Thriller|       avg(rating)|
+--------+------------------+
|       1|3.5090069495245064|
+--------+------------------+



In [53]:
df_war = df.join(df_movie, df.movie_id==df_movie.movie_id, 'inner').select(df_movie.War, df.rating)
df_war = df_war.groupBy('War').mean('rating')
df_war.filter(df_war.War == 1).show()

+---+-----------------+
|War|      avg(rating)|
+---+-----------------+
|  1|3.815811874866993|
+---+-----------------+



In [54]:
df_western = df.join(df_movie, df.movie_id==df_movie.movie_id, 'inner').select(df_movie.Western, df.rating)
df_western = df_western.groupBy('Western').mean('rating')
df_western.filter(df_western.Western == 1).show()

+-------+------------------+
|Western|       avg(rating)|
+-------+------------------+
|      1|3.6132686084142396|
+-------+------------------+



##В спарке получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок, либо по самой оценке - на Ваш выбор)

In [55]:
df = spark.read.csv('ml-100k/u.data', inferSchema=True, header=False, sep='\t')
df = df.withColumnRenamed("_c0", "user_id").withColumnRenamed("_c1", "movie_id") \
        .withColumnRenamed("_c2", "rating").withColumnRenamed("_c3", "timestamp")
df.show(5)

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|    196|     242|     3|881250949|
|    186|     302|     3|891717742|
|     22|     377|     1|878887116|
|    244|      51|     2|880606923|
|    166|     346|     1|886397596|
+-------+--------+------+---------+
only showing top 5 rows



In [68]:
df_top = df.groupBy('movie_id').count()
df_top.show(5)

+--------+-----+
|movie_id|count|
+--------+-----+
|     496|  231|
|     471|  221|
|     463|   71|
|     148|  128|
|    1342|    2|
+--------+-----+
only showing top 5 rows



In [73]:
df_top.sort(df_top['count'].desc()).show(5)

+--------+-----+
|movie_id|count|
+--------+-----+
|      50|  583|
|     258|  509|
|     100|  508|
|     181|  507|
|     294|  485|
+--------+-----+
only showing top 5 rows



In [75]:
df_top.sort(df_top['count']).show()

+--------+-----+
|movie_id|count|
+--------+-----+
|    1352|    1|
|    1525|    1|
|    1580|    1|
|    1645|    1|
|    1507|    1|
|    1460|    1|
|    1339|    1|
|    1650|    1|
|    1618|    1|
|     857|    1|
|    1533|    1|
|    1653|    1|
|    1561|    1|
|    1493|    1|
|    1675|    1|
|     830|    1|
|    1201|    1|
|    1510|    1|
|    1621|    1|
|    1156|    1|
+--------+-----+
only showing top 20 rows

